adbot

In [5]:
import pandas as pd
import numpy as np
from datetime import timedelta
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split  # Added import
from sklearn.ensemble import RandomForestRegressor

# Read the data
df = pd.read_csv("Train (1).csv")

# Convert 'date' column to datetime format
df['date'] = pd.to_datetime(df['date'])

# Drop unnecessary columns
cols_to_drop = ['cost', 'conversions', 'currency', 'call_type', 'call_status', 
                'start_time', 'duration', 'end_time', 'impression_share', 
                'display_location', 'conversions_calls']
df = df.drop(cols_to_drop, axis=1)
df.dropna(inplace=True)

# Perform one-hot encoding for the 'ad_type' column
df_encoded = pd.get_dummies(df, columns=['ad_type'])

# Ordinal encode the 'ID' column
label_encoder = LabelEncoder()
df_encoded['ID_encoded'] = label_encoder.fit_transform(df_encoded['ID'])

# Split the data into features and target
X = df_encoded.drop(['clicks', 'date', 'ID'], axis=1)
y = df_encoded['clicks']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Train the RandomForestRegressor model
forest = RandomForestRegressor()
forest.fit(X_train, y_train)

# Calculate the score of the model on the test set
print("Model Score on Test Data:", forest.score(X_test, y_test))



Model Score on Test Data: 0.970721986692287


In [6]:
# Generate future dates for one and two weeks ahead
future_date_1_week = pd.Timestamp.now() + timedelta(days=7)
future_date_2_weeks = pd.Timestamp.now() + timedelta(days=14)

# Create a DataFrame for one and two weeks future dates
future_df_1_week = pd.DataFrame({'date': [future_date_1_week] * len(unique_ids), 'ID': unique_ids})
future_df_2_weeks = pd.DataFrame({'date': [future_date_2_weeks] * len(unique_ids), 'ID': unique_ids})


# Filter data based on unique 'ID' values
unique_ids = df_encoded['ID'].unique()
future_df_1_week['ID'] = unique_ids
future_df_2_weeks['ID'] = unique_ids

# Ordinal encode the 'ID' column in future data
future_df_1_week['ID_encoded'] = label_encoder.transform(future_df_1_week['ID'])
future_df_2_weeks['ID_encoded'] = label_encoder.transform(future_df_2_weeks['ID'])

# Get the feature names used during training
feature_names = X.columns.tolist()

# Filter the future DataFrames to include only the relevant features
future_df_1_week = future_df_1_week[['ID', 'date'] + feature_names]
future_df_2_weeks = future_df_2_weeks[['ID', 'date'] + feature_names]

# Make predictions for one and two weeks future dates
future_clicks_pred_1_week = forest.predict(future_df_1_week.drop(['date', 'ID'], axis=1))
future_clicks_pred_2_weeks = forest.predict(future_df_2_weeks.drop(['date', 'ID'], axis=1))

# Inverse transform the predictions
future_clicks_pred_1_week_inv = np.exp(future_clicks_pred_1_week) - 1
future_clicks_pred_2_weeks_inv = np.exp(future_clicks_pred_2_weeks) - 1

# Display the predictions
print("\nPredicted clicks for one week ahead:")
print(future_clicks_pred_1_week_inv)

print("\nPredicted clicks for two weeks ahead:")
print(future_clicks_pred_2_weeks_inv)


KeyError: "['impressions', 'headline1_len', 'headline2_len', 'ad_description_len', 'ad_type_EXPANDED_DYNAMIC_SEARCH_AD', 'ad_type_EXPANDED_TEXT_AD', 'ad_type_RESPONSIVE_SEARCH_AD'] not in index"